# 1.Autoencoders

## Build different architectures

Archtecture 1

In [ ]:
class Autoencoder_p2_v1(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(36, 15),
            nn.ReLU(),
            nn.Linear(15, 7),
            nn.ReLU(),
            nn.Linear(7, 2)
        )

        self.decoder = nn.Sequential(
            nn.Linear(2, 7),
            nn.ReLU(),
            nn.Linear(7, 15),
            nn.ReLU(),
            nn.Linear(15, 36)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

Architecture 2

In [ ]:
class Autoencoder_p2_v2(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(36, 26),
            nn.ReLU(),
            nn.Linear(26, 15),
            nn.ReLU(),
            nn.Linear(15, 7),
            nn.ReLU(),
            nn.Linear(7, 3),
        )

        self.decoder = nn.Sequential(
            nn.Linear(3, 7),
            nn.ReLU(),
            nn.Linear(7, 15),
            nn.ReLU(),
            nn.Linear(15, 26),
            nn.ReLU(),
            nn.Linear(26, 36)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

## Initiate the model an set up loss function, optimizer, scheduler

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# MSE loss
loss_fn = nn.MSELoss().to(device)

# Adam optimizer
optimizer = optim.Adam(model_autoencoder_p2_v2.parameters())

# Use scheduler to automatically adjust learning rate
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.2, verbose=True)

model_autoencoder_p2_v2 = Autoencoder_p2_v2()
model_autoencoder_p2_v2 = model_autoencoder_p2_v2.to(device)

## Set up batch size and create Dataloader

In [ ]:
BATCH_SIZE = 2048
train_loader = DataLoader(dataset=x_train_p2, shuffle=True, batch_size=BATCH_SIZE)
val_loader = DataLoader(dataset=x_val_p2, shuffle=True, batch_size=BATCH_SIZE)

## Model Training

In [ ]:
model_name = model_autoencoder_p2_v1

import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

n_epochs = 50
history = dict(train=[], val=[])
best_val_loss = float('inf')

for epoch in range(n_epochs):
    start_time = time.time()

    train_loss = train_epoch(model_name, train_loader, optimizer, loss_fn)
    val_loss = eval_epoch(model_name, val_loader, loss_fn)
    history['train'].append(train_loss)
    history['val'].append(val_loss)
    scheduler.step(val_loss)
    end_time =time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model_name.state_dict(), 'model_name.pth')

    print(f'Epoch: {epoch + 1:02} |Epoch Time:{epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss: .3f} ')
    print(f'\tVal Loss: {val_loss: .3f} ')

## Plot training and valiadation losses

In [ ]:
ax = plt.figure().gca()
ax.plot(history['train'])
ax.plot(history['val'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.title('Loss over training epochs')
plt.show()

# 2. One-Class SVM

In [ ]:
# Import SVM packages from sklearn
from sklearn.svm import OneClassSVM

## SVMs with different hyperparameters.

In [ ]:
SVM_rbf_v1 = OneClassSVM(kernel='rbf', nu=0.01, gamma=0.1)

SVM_rbf_v2 = OneClassSVM(kernel='rbf', nu=0.1, gamma=0.1)

SVM_rbf_v3 = OneClassSVM(kernel='rbf', nu=0.001, gamma=0.1)

# 3. Isolation Forests

In [ ]:
from sklearn.ensemble import IsolationForest

## Isolation Forests with different contaminations ratios.

In [ ]:
clf1 = IsolationForest(max_samples=100, random_state=42, contamination=0.01)

clf2 = IsolationForest(max_samples=100, random_state=42, contamination=0.05)

clf3 = IsolationForest(max_samples=100, random_state=42, contamination=0.1)

fit the models on training set

In [ ]:
clf.fit(x_train)